In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [2]:
fact_df = pd.read_csv('data/fact_table.csv', encoding='ISO-8859-1')
customer_df = pd.read_csv('data/customer_dim.csv', encoding='ISO-8859-1')
item_df = pd.read_csv('data/item_dim.csv', encoding='ISO-8859-1')
store_df = pd.read_csv('data/store_dim.csv', encoding='ISO-8859-1')
time_df = pd.read_csv('data/time_dim.csv', encoding='ISO-8859-1')

In [3]:
df = (
    fact_df.merge(time_df, how='left', on='time_key')
    .merge(item_df, how='left', on='item_key')
    .merge(customer_df, how='left', on='coustomer_key')
)

In [4]:
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M')
df['minute'] = df['date'].dt.minute
df['date'] = pd.to_datetime(df['date'].dt.date)

In [5]:
product_journey_df = df.sort_values(by=['coustomer_key', 'date'])
product_journey_df['next_item_bought'] = product_journey_df.groupby('coustomer_key')['item_name'].shift(-1)
product_journey_df = product_journey_df.dropna(subset=['next_item_bought']).reset_index()
product_journey_df = product_journey_df[['coustomer_key','item_key','date','quantity','total_price','item_name','next_item_bought']]
product_journey_df

,coustomer_key,item_key,date,quantity,total_price,item_name,next_item_bought
0,C000001,I00137,2014-02-06,11,264.0,Foam Coffee Cups - 08 ounce,Pringles Snak Stacks Variety
1,C000001,I00167,2014-02-21,7,98.0,Pringles Snak Stacks Variety,Nature Valley Biscuit Sandwich Variety Pack
2,C000001,I00204,2014-02-22,6,102.0,Nature Valley Biscuit Sandwich Variety Pack,K Cups - McCafe Premium Roast
3,C000001,I00113,2014-04-03,2,84.0,K Cups - McCafe Premium Roast,Foam Coffee Cups - 16 ounce
4,C000001,I00139,2014-05-18,7,168.0,Foam Coffee Cups - 16 ounce,Tylenol Extra Strength 2 pill packets
...,...,...,...,...,...,...,...
990804,C009191,I00118,2020-11-01,3,126.0,K Cups Organic Breakfast Blend,K Cups Organic Breakfast Blend
990805,C009191,I00118,2020-11-30,6,252.0,K Cups Organic Breakfast Blend,Belvita Hard Biscuits Blueberry
990806,C009191,I00186,2020-12-06,2,26.0,Belvita Hard Biscuits Blueberry,Large Trash Bags Black 33 gal
990807,C009191,I00257,2020-12-17,6,126.0,Large Trash Bags Black 33 gal,Pepsi - 12 oz cans


In [6]:
all_products = list(set(product_journey_df['item_name']).union(set(product_journey_df['next_item_bought'])))

# Create a mapping from product name to index
product_to_index = {product: i for i, product in enumerate(all_products)}

# Create source and target indices
product_journey_df['source'] = product_journey_df['item_name'].map(product_to_index)
product_journey_df['target'] = product_journey_df['next_item_bought'].map(product_to_index)

# Aggregate data for Sankey diagram
sankey_data = product_journey_df.groupby(['source', 'target']).size().reset_index(name='value').sort_values('value', ascending=False).iloc[:25]

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=[product for product in all_products]
    ),
    link=dict(
        source=sankey_data['source'],
        target=sankey_data['target'],
        value=sankey_data['value']
    )
)])

fig.update_layout(title_text="Customer Journey of Items Purchased", font_size=10)
fig.show()

In [7]:
from efficient_apriori import apriori

basket = df.groupby(['coustomer_key', 'item_name'])['quantity'].sum().unstack().reset_index().fillna(0).set_index('coustomer_key')
min_purchase_threshold = 23000
min_customer_threshold = 650

product_counts = basket.sum(axis=0)
filtered_products = product_counts[product_counts >= min_purchase_threshold].index
basket_filtered = basket[filtered_products]

customer_sums = basket.sum(axis=1)
filtered_customers = customer_sums[customer_sums >= min_customer_threshold].index
basket_filtered = basket_filtered.loc[filtered_customers]

basket = basket_filtered.applymap(lambda x: 1 if x > 4 else 0)

In [8]:
basket.shape

(4691, 64)

In [9]:
transactions = basket.apply(lambda row: [product for product in basket.columns if row[product] == 1], axis=1).tolist()

itemsets, rules = apriori(transactions, min_support=0.05, min_confidence=0.5)

In [10]:
itemsets[3]

{('100% Juice Box Variety 6.75 oz ',
  'Coke Classic 12 oz cans',
  'Diet Coke - 12 oz cans'): 255,
 ('100% Juice Box Variety 6.75 oz ',
  'Coke Classic 12 oz cans',
  'Muscle Milk Protein Shake Van. 11oz'): 248,
 ('100% Juice Box Variety 6.75 oz ',
  'Coke Classic 12 oz cans',
  'Pepsi - 12 oz cans'): 259,
 ('100% Juice Box Variety 6.75 oz ',
  'Coke Classic 12 oz cans',
  'Sprite - 12 oz cans'): 254,
 ('100% Juice Box Variety 6.75 oz ',
  'Diet Coke - 12 oz cans',
  'Muscle Milk Protein Shake Van. 11oz'): 240,
 ('100% Juice Box Variety 6.75 oz ',
  'Diet Coke - 12 oz cans',
  'Pepsi - 12 oz cans'): 255,
 ('100% Juice Box Variety 6.75 oz ',
  'Diet Coke - 12 oz cans',
  'Sprite - 12 oz cans'): 242,
 ('100% Juice Box Variety 6.75 oz ',
  'Muscle Milk Protein Shake Van. 11oz',
  'Pepsi - 12 oz cans'): 254,
 ('100% Juice Box Variety 6.75 oz ',
  'Muscle Milk Protein Shake Van. 11oz',
  'Sprite - 12 oz cans'): 236,
 ('100% Juice Box Variety 6.75 oz ',
  'Pepsi - 12 oz cans',
  'Sprite - 1

In [11]:
len(rules)

9

In [12]:
top_itemsets = itemsets.nlargest(10, 'support')
top_itemsets['itemsets'] = top_itemsets['itemsets'].apply(lambda x: ', '.join(x))

plt.figure(figsize=(10, 6))
plt.barh(top_itemsets['itemsets'], top_itemsets['support'], color='skyblue')
plt.xlabel('Support')
plt.ylabel('Itemsets')
plt.title('Top 10 Frequent Itemsets')
plt.gca().invert_yaxis()
plt.show()

AttributeError: 'dict' object has no attribute 'nlargest'